In [12]:
import numpy as np
import os
import io
import cv2

import torch

from config import (Config,
                    eval_dict_leaf)

from utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

In [13]:
video = cv2.VideoCapture('/mnt/xingjian_luo/dataset/CholecT50/video_clips_short/VID56_107.mp4')
frames = [x for x in _frame_from_video(video)]

In [14]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "The endoscopic video depicts a laparoscopic cholecystectomy procedure. During the \"preparation\" phase, a grasper surgical instrument is actively retracting the gallbladder. The grasper's position varies across the frames, being located around the center, moving closer to the top-right, and making dynamic adjustments throughout the section.",
                   "The video depicts a laparoscopic cholecystectomy surgery during the calot-triangle-dissection phase. Throughout the video, a grasper is frequently seen retracting the gallbladder, while a hook engages in dissection of the gallbladder. The grasper is consistently positioned near the lower-center of the field, and the hook's location is not within the visible field. The surgical instruments are actively engaged in these tasks, and the procedures smoothly progress without any idle time noted for the instruments.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "The endoscopic video features a laparoscopic cholecystectomy surgery in its preparation phase. Throughout the footage, a grasper instrument is consistently seen retracting the gallbladder. Initially, the grasper is located near the lower-right area of the screen, later moving towards the center. In subsequent frames, the exact location of the grasper in the frame becomes unclear.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

In [15]:
config = Config.from_file('/mnt/xingjian_luo/project/videollm/internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [16]:
model_pth = '/mnt/xingjian_luo/project/videollm/multi_modality/scripts/pretraining/stage2/1B/run.sh_20240809_055716/ckpt_latest/mp_rank_00_model_states.pt'
config['pretrained_path'] = model_pth
config["model"]["vision_encoder"]["pretrained"] = model_pth

In [7]:
config.model.text_encoder["config"] = "/mnt/xingjian_luo/project/InternVideo/InternVideo2/multi_modality/configs/config_bert_large.json"

In [8]:
config.model.text_encoder["pretrained"] = "/mnt/xingjian_luo/checkpoint/bert-large-uncased"

In [9]:
config

{'available_corpus': {'cc3m': {'anno_path': 'your_path',
   'data_root': '',
   'media_type': 'image'},
  'cc12m': {'anno_path': 'your_path', 'data_root': '', 'media_type': 'image'},
  'sbu': {'anno_path': 'your_path', 'data_root': '', 'media_type': 'image'},
  'vg': {'anno_path': 'your_path',
   'data_root': '',
   'media_type': 'image',
   'jump_filter': True},
  'coco': {'anno_path': 'your_path',
   'data_root': '',
   'media_type': 'image',
   'jump_filter': True},
  'laion_2b': {'anno_path': 'your_path',
   'data_root': '',
   'media_type': 'image',
   'jump_filter': True},
  'laion_coco': {'anno_path': 'your_path',
   'data_root': '',
   'media_type': 'image',
   'jump_filter': True},
  'laion_pop': {'anno_path': 'your_path',
   'data_root': '',
   'media_type': 'image',
   'jump_filter': True},
  'cholect50': {'anno_path': '/mnt/xingjian_luo/dataset/cholect50-challenge-val/stage2_cholec50_caption.json',
   'data_root': '',
   'media_type': 'video',
   'jump_filter': True},
  'ch

In [10]:
intern_model, tokenizer = setup_internvideo2(config)

load_state_dict: _IncompatibleKeys(missing_keys=[], unexpected_keys=['temp', 'itm_head.weight', 'itm_head.bias'])


In [17]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

text: The endoscopic video depicts a laparoscopic cholecystectomy procedure. During the "preparation" phase, a grasper surgical instrument is actively retracting the gallbladder. The grasper's position varies across the frames, being located around the center, moving closer to the top-right, and making dynamic adjustments throughout the section. ~ prob: 0.9080
text: The endoscopic video features a laparoscopic cholecystectomy surgery in its preparation phase. Throughout the footage, a grasper instrument is consistently seen retracting the gallbladder. Initially, the grasper is located near the lower-right area of the screen, later moving towards the center. In subsequent frames, the exact location of the grasper in the frame becomes unclear. ~ prob: 0.0713
text: The video depicts a laparoscopic cholecystectomy surgery during the calot-triangle-dissection phase. Throughout the video, a grasper is frequently seen retracting the gallbladder, while a hook engages in dissection of the gallb